#### [Leetcode 773 Hard] [Sliding Puzzle](https://leetcode.com/problems/sliding-puzzle/)

On a 2x3 board, there are 5 tiles represented by the integers 1 through 5, and an empty square represented by 0.

A move consists of choosing 0 and a 4-directionally adjacent number and swapping it.

The state of the board is solved if and only if the board is [[1,2,3],[4,5,0]].

Given a puzzle board, return the least number of moves required so that the state of the board is solved. If it is impossible for the state of the board to be solved, return -1.

Examples:
```
Input: board = [[1,2,3],[4,0,5]]
Output: 1
Explanation: Swap the 0 and the 5 in one move.
Input: board = [[1,2,3],[5,4,0]]
Output: -1
Explanation: No number of moves will make the board solved.
Input: board = [[4,1,2],[5,0,3]]
Output: 5
Explanation: 5 is the smallest number of moves that solves the board.
An example path:
After move 0: [[4,1,2],[5,0,3]]
After move 1: [[4,1,2],[0,5,3]]
After move 2: [[0,1,2],[4,5,3]]
After move 3: [[1,0,2],[4,5,3]]
After move 4: [[1,2,0],[4,5,3]]
After move 5: [[1,2,3],[4,5,0]]
Input: board = [[3,2,4],[1,5,0]]
Output: 14
```

Note:
* board will be a 2 x 3 array as described above.
* board[i][j] will be a permutation of [0, 1, 2, 3, 4, 5].

Hint:
1. Perform a breadth-first-search, where the nodes are the puzzle boards and edges are if two puzzle boards can be transformed into one another with one move.

<font color='red'>IMPORTANT: </font> read this [Introduction to the A\* Algorithm](https://www.redblobgames.com/pathfinding/a-star/introduction.html)

<font color='blue'>Solution: </font>: BFS
* Time Complexity: O(n!), n = row \* col
* Space Complexity: O(n)

使用最普通的BFS遍历方式, 就检查上下左右四个方向，那么这样我们就不能压缩二维数组成一个字符串了，我们visited数组中只能放二维数组了，同样的，queue 中也只能放二维数组，由于二维数组要找0的位置的话，还需要遍历，为了节省遍历时间，我们将0的位置也放入queue中，那么queue中的放的就是一个pair对儿，保存当前状态，已经0的位置，初始时将棋盘以及0的位置排入queue中。之后的操作就跟之前的解法没啥区别了，只不过这里我们的心位置就是上下左右，如果未越界的话，那么和0交换位置，看新状态是否已经出现过，如果这个状态不在visited中，则加入visited，并且压入队列queue，步数自增1。如果while循环退出后都没有回到正确状态，则返回-1

In [1]:
class Solution(object):
    def slidingPuzzle(self, board):
        """
        :type board: List[List[int]]
        :rtype: int
        """
        # results = []
        count = 0
        
        frontier = []
        visited = set()
        
        row = len(board)
        col = len(board[0])
        
        # set up the target
        # in python, list is not hashtable
        # hence we use a string
        target_board = [str(i) for i in range(1, row * col + 1)]
        target_board[-1] = str(0)
        target = "".join(target_board)
        
        # set up the start string
        start = self.board2str(board)
        
        # seek the starting point: the location of 0  
        for i in range(row):
            for j in range(col):
                if board[i][j] == 0:
                    frontier.append((start, i, j))                  
        
        
        dx, dy = [-1, 0, 1, 0], [0, -1, 0, 1]
        while frontier:
            post = []
            for grid, x, y in frontier:  # get the x, y position of a node
                if grid == target:
                    return count
                
                for d in range(4):
                    nx, ny = x + dx[d], y + dy[d]
                    if ((0 <= nx < row) and (0 <= ny < col)):
                        # update the grid
                        new_grid = list(grid)
                        new_grid[x * col + y], new_grid[nx * col + ny] = new_grid[nx * col + ny], new_grid[x * col + y]
                        new_grid = "".join(new_grid)
                        if new_grid not in visited:
                            visited.add(new_grid)
                            post.append((new_grid, nx, ny))

            count += 1
            frontier = post
                
        return -1
    
    def board2str(self, board):
        arr = []
        for i in range(len(board)):
            for j in range(len(board[0])):
                arr.append(str(board[i][j]))
        
        return "".join(arr)

In [2]:
soln = Solution()
print(soln.slidingPuzzle( board=[[1,2,3],[4,0,5]]))
print(soln.slidingPuzzle( board=[[1,2,3],[5,4,0]]))
print(soln.slidingPuzzle( board=[[4,1,2],[5,0,3]]))
print(soln.slidingPuzzle( board=[[3,2,4],[1,5,0]]))

1
-1
5
14


#### Follow up 2: Output: procedure


In [13]:
class Solution(object):
    def slidingPuzzle(self, board):
        """
        :type board: List[List[int]]
        :rtype: int
        """
        # results = []
        count = 0
        
        frontier = []
        came_from = {}
        
        row = len(board)
        col = len(board[0])
        
        # set up the target
        # in python, list is not hashtable
        # hence we use a string
        target_board = [str(i) for i in range(1, row * col + 1)]
        target_board[-1] = str(0)
        target = "".join(target_board)
        
        # set up the start string
        start = self.board2str(board)
        
        # seek the starting point: the location of 0  
        for i in range(row):
            for j in range(col):
                if board[i][j] == 0:
                    frontier.append((start, i, j))
                    
        came_from[start] = None        
        
        dx, dy = [-1, 0, 1, 0], [0, -1, 0, 1]
        while frontier:
            post = []
            for grid, x, y in frontier:  # get the current state
                if grid == target:
                    # construct the path
                    path = []
                    current = target
                    while current != start:
                        path.append(current)
                        current = came_from[current]
                    path.append(start)
                    path = path[:]
                    #print(count, path)
                    return count, path
                
                for d in range(4):
                    nx, ny = x + dx[d], y + dy[d]
                    if ((0 <= nx < row) and (0 <= ny < col)):
                        # update the grid
                        new_grid = list(grid)
                        new_grid[x * col + y], new_grid[nx * col + ny] = new_grid[nx * col + ny], new_grid[x * col + y]
                        new_grid = "".join(new_grid)
                        if new_grid not in came_from:
                            came_from[new_grid] = grid
                            post.append((new_grid, nx, ny)) # add the next state

            count += 1
            frontier = post
                
        return -1, None
    
    def board2str(self, board):
        arr = []
        for i in range(len(board)):
            for j in range(len(board[0])):
                arr.append(str(board[i][j]))
        
        return "".join(arr)

In [14]:
soln = Solution()
res = soln.slidingPuzzle(board=[[4,1,2],[5,0,3]])
print(res)

(5, ['123450', '120453', '102453', '012453', '412053', '412503'])


#### Follow up 3: Optimization by A\*

In [40]:
import heapq

class Solution(object):
    def slidingPuzzle(self, board):
        """
        :type board: List[List[int]]
        :rtype: int
        """       
        row = len(board)
        col = len(board[0])
        
        frontier = []
        # set up the start string
        start = self.board2str(board)        
        # seek the starting point: the location of 0  
        for i in range(row):
            for j in range(col):
                if board[i][j] == 0:
                    heapq.heappush(frontier, (0, 0 ,start, i, j)) # cost, count, node, x, y
                                        
        came_from = {}
        came_from[start] = (None, float('inf'))
        
        cost_so_far = {}
        cost_so_far[start] = 0

        # set up the target
        # in python, list is not hashtable
        # hence we use a string
        target_board = [str(i) for i in range(1, row * col + 1)]
        target_board[-1] = str(0)
        target = "".join(target_board)      
    
        dx, dy = [-1, 0, 1, 0], [0, -1, 0, 1]
        while frontier:
            curr_cost, curr_count, curr_grid, x, y = heapq.heappop(frontier)
            if curr_grid == target:
                # construct the path
                path = []
                current, priority = target, 0
                while current != start:
                    path.append((current, priority))
                    current, priority = came_from[current]
                path.append(start)
                path = path[::-1]
                #print(count, path)
                return curr_count, path

            for d in range(4):
                nx, ny = x + dx[d], y + dy[d]
                if ((0 <= nx < row) and (0 <= ny < col)):
                    # update the grid
                    next_grid = list(curr_grid)
                    next_grid[x * col + y], next_grid[nx * col + ny] = next_grid[nx * col + ny], next_grid[x * col + y]
                    next_grid = "".join(next_grid)
                    
                    next_cost = cost_so_far[curr_grid]
                    if next_grid not in cost_so_far or next_cost < cost_so_far[next_grid]:
                        priority = next_cost + self.get_cost(next_grid, target, col)
                        cost_so_far[next_grid] = priority                        
                        heapq.heappush(frontier, (priority, curr_count + 1, next_grid, nx, ny)) # add the next state
                        #print(frontier)
                        came_from[next_grid] = (curr_grid, priority)
               
        return -1, None
    
    def board2str(self, board):
        arr = []
        for i in range(len(board)):
            for j in range(len(board[0])):
                arr.append(str(board[i][j]))
        
        return "".join(arr)
    
    def get_cost(self, string, target, col):
        total_cost = 0
        
        for curr_index, char in enumerate(string):
            curr_row, curr_col = curr_index // col,  curr_index % col
            target_index = target.index(char)
            target_row, target_col = target_index // col,  target_index % col
            cost = abs(target_row - curr_row) + abs(target_col - curr_col)
            total_cost += cost
            
        return total_cost

In [41]:
soln = Solution()
print(soln.slidingPuzzle( board=[[1,2,3],[4,0,5]]))
print(soln.slidingPuzzle( board=[[1,2,3],[5,4,0]]))
print(soln.slidingPuzzle( board=[[4,1,2],[5,0,3]]))
print(soln.slidingPuzzle( board=[[3,2,4],[1,5,0]]))

(1, ['123405', ('123450', 0)])
(-1, None)
(5, ['412503', ('412053', 12), ('012453', 16), ('102453', 18), ('120453', 18), ('123450', 0)])
(14, ['324150', ('324105', 18), ('304125', 26), ('340125', 34), ('345120', 44), ('345102', 54), ('305142', 64), ('035142', 72), ('135042', 78), ('135402', 82), ('135420', 86), ('130425', 90), ('103425', 92), ('123405', 92), ('123450', 0)])


#### Follow up: How to determine whether the solution exists

可以在O((nm)^2)或是O(nm lg(nm))的时间根据奇偶性判断 [Source](https://www.cs.bham.ac.uk/~mdr/teaching/modules04/java2/TilesSolvability.html)